In [19]:
import pandas as pd
import numpy as np
from scipy import stats

# Load Europe prices monthly data
df1 = pd.read_csv('europe indices monthly.csv')

# Load German bond data
df_bond = pd.read_csv('german bond.csv')
# Convert date columns to consistent format
df1['Date'] = pd.to_datetime(df1['Date'])
df_bond['Date'] = pd.to_datetime(df_bond['Date'])

# Set the date columns as the DataFrame index
df1.set_index('Date', inplace=True)
df_bond.set_index('Date', inplace=True)

# Calculate monthly returns for each index
returns = df1.pct_change().dropna()

# Calculate excess returns over German bond
df1_excess = returns.sub(df_bond['GER 10Y Bond'], axis='rows')

# Calculate annualized mean return and standard deviation
mean_returns = df1_excess.mean() * 12
std_returns = df1_excess.std() * np.sqrt(12)

# Calculate risk-free rate (assumed to be 0 in this example)
risk_free_rate = -0.0002

# Calculate Sharpe ratio
sharpe_ratio = (mean_returns - risk_free_rate) / std_returns

# Create a DataFrame to store the Sharpe ratio results
df_sharpe_ratio = pd.DataFrame({'Index': sharpe_ratio.index, 'Sharpe Ratio': sharpe_ratio})

# Sort the DataFrame by Sharpe ratio in descending order
df_sharpe_ratio = df_sharpe_ratio.sort_values(by='Sharpe Ratio', ascending=False)

# Print the results
print(df_sharpe_ratio)


           Index  Sharpe Ratio
DAX          DAX      0.547393
CAC 40    CAC 40      0.364615
EUSTX50  EUSTX50      0.254813
IT40        IT40      0.156823
IBEX 35  IBEX 35     -0.034949


In [39]:
market_returns=pd.read_csv('europe benchmark.csv')
# Define the beta values for each index
beta_values = {
    'EUSTX50': 0.85,
    'CAC 40': 0.82,
    'IBEX 35': 0.95,
    'DAX': 0.80,
    'IT40': 1.01
}




# Create a DataFrame to store the Jensen's alpha results
df_jensen_alpha = pd.DataFrame(columns=['Index', 'Jensen Alpha'])

# Iterate over each index and calculate Jensen's alpha
for index in df1_excess.columns:
    if index != 'Jensen Alpha':
        beta = beta_values[index]
        expected_returns = risk_free_rate + beta * (market_returns['MSCI'] - risk_free_rate)
        alpha = df1_excess[index].sub(expected_returns, fill_value=0).mean()
        df_jensen_alpha = df_jensen_alpha.append({'Index': index, 'Jensen Alpha': alpha}, ignore_index=True)

# Sort the DataFrame by Jensen's alpha in descending order
df_jensen_alpha = df_jensen_alpha.sort_values(by='Jensen Alpha', ascending=False)

# Print the results
print(df_jensen_alpha)


     Index Jensen Alpha
3      DAX  -639.245899
1   CAC 40  -655.228603
0  EUSTX50  -679.201207
2  IBEX 35  -759.109568
4     IT40   -807.05167


C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\2213983273.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_jensen_alpha = df_jensen_alpha.append({'Index': index, 'Jensen Alpha': alpha}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\2213983273.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_jensen_alpha = df_jensen_alpha.append({'Index': index, 'Jensen Alpha': alpha}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\2213983273.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_jensen_alpha = df_jensen_alpha.append({'Index': index, 'Jensen Alpha': alpha}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\2213983273.py:23: 

In [26]:
# Create a DataFrame to store the Treynor ratio results
df_treynor_ratio = pd.DataFrame(columns=['Index', 'Treynor Ratio'])

# Iterate over each index and calculate the Treynor ratio
for index in df1_excess.columns:
    if index != 'Jensen Alpha':
        beta = beta_values[index]
        treynor_ratio = df1_excess[index].mean() / beta
        df_treynor_ratio = df_treynor_ratio.append({'Index': index, 'Treynor Ratio': treynor_ratio}, ignore_index=True)

# Sort the DataFrame by Treynor ratio in descending order
df_treynor_ratio = df_treynor_ratio.sort_values(by='Treynor Ratio', ascending=False)

# Print the results
print(df_treynor_ratio)


     Index Treynor Ratio
3      DAX      0.010955
1   CAC 40      0.007151
0  EUSTX50      0.005209
4     IT40      0.003609
2  IBEX 35      0.000298


C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\2406351413.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_treynor_ratio = df_treynor_ratio.append({'Index': index, 'Treynor Ratio': treynor_ratio}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\2406351413.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_treynor_ratio = df_treynor_ratio.append({'Index': index, 'Treynor Ratio': treynor_ratio}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\2406351413.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_treynor_ratio = df_treynor_ratio.append({'Index': index, 'Treynor Ratio': treynor_ratio}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\ipy

In [27]:
# Define the target return or minimum acceptable return
target_return = 0

# Create a DataFrame to store the Sortino ratio results
df_sortino_ratio = pd.DataFrame(columns=['Index', 'Sortino Ratio'])

# Iterate over each index and calculate the Sortino ratio
for index in df1_excess.columns:
    if index != 'Jensen Alpha':
        excess_returns = df1_excess[index]
        downside_returns = excess_returns[excess_returns < target_return]
        downside_deviation = downside_returns.std()
        
        if downside_deviation == 0:
            sortino_ratio = np.inf  # Handle zero downside deviation case
        else:
            sortino_ratio = excess_returns.mean() / downside_deviation
        
        df_sortino_ratio = df_sortino_ratio.append({'Index': index, 'Sortino Ratio': sortino_ratio}, ignore_index=True)

# Sort the DataFrame by Sortino ratio in descending order
df_sortino_ratio = df_sortino_ratio.sort_values(by='Sortino Ratio', ascending=False)

# Print the results
print(df_sortino_ratio)


     Index Sortino Ratio
3      DAX      0.243212
1   CAC 40      0.188416
0  EUSTX50      0.140486
4     IT40      0.086505
2  IBEX 35      0.007206


C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\4017336777.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sortino_ratio = df_sortino_ratio.append({'Index': index, 'Sortino Ratio': sortino_ratio}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\4017336777.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sortino_ratio = df_sortino_ratio.append({'Index': index, 'Sortino Ratio': sortino_ratio}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\ipykernel_7460\4017336777.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sortino_ratio = df_sortino_ratio.append({'Index': index, 'Sortino Ratio': sortino_ratio}, ignore_index=True)
C:\Users\cdimopoulos\AppData\Local\Temp\

In [51]:
import pandas as pd

# Define the risk-free rate
risk_free_rate = 0.03

# Read the market returns data
market_returns = pd.read_csv('europe benchmark.csv')

# Define the beta values for each index
beta_values = {
    'EUSTX50': 0.85,
    'CAC 40': 0.82,
    'IBEX 35': 0.95,
    'DAX': 0.80,
    'IT40': 1.01
}

# Create a DataFrame to store the Jensen's alpha results
df_jensen_alpha = pd.DataFrame(columns=['Index', 'Jensen Alpha'])

# Select the benchmark returns column
benchmark_returns = market_returns.iloc[:, 1]

# Iterate over each index and calculate Jensen's alpha
for index in market_returns.columns[2:]:
    beta = beta_values[index]
    expected_returns = risk_free_rate + beta * (benchmark_returns - risk_free_rate)
    alpha = market_returns[index].sub(expected_returns, fill_value=0).mean()
    df_jensen_alpha = df_jensen_alpha.append({'Index': index, 'Jensen Alpha': alpha}, ignore_index=True)

# Drop rows with missing values
df_jensen_alpha.dropna(inplace=True)

# Sort the DataFrame by Jensen's alpha in descending order
df_jensen_alpha = df_jensen_alpha.sort_values(by='Jensen Alpha', ascending=False)

# Print the results
print(df_jensen_alpha)



Empty DataFrame
Columns: [Index, Jensen Alpha]
Index: []
